In [2]:
!pip install -q kaggle
!pip install opendatasets

In [3]:
import os
import zipfile
import shutil
import random
import numpy as np
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img, img_to_array, array_to_img, save_img
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from google.colab import files
import opendatasets as od

In [6]:
files.upload()

In [5]:
od.download(
    "https://www.kaggle.com/datasets/emmarex/plantdisease")

Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease


100%|██████████| 658M/658M [00:08<00:00, 83.8MB/s]


In [7]:
original_dir = '/content/plantdisease/plantvillage/PlantVillage'
split_dir = '/content/plantvillage_split'
split_ratio = 0.2
IMG_SIZE = (128, 128)

os.makedirs(split_dir, exist_ok=True)
for split in ['train', 'val']:
    os.makedirs(os.path.join(split_dir, split), exist_ok=True)

def random_orient_augment(img_path, img_size):
    img = load_img(img_path, target_size=img_size)
    img_arr = img_to_array(img)
    if random.random() > 0.5:
        img_arr = tf.image.flip_left_right(img_arr)
    if random.random() > 0.5:
        img_arr = tf.image.flip_up_down(img_arr)
    k = random.randint(0, 3)
    img_arr = tf.image.rot90(img_arr, k=k)
    return array_to_img(img_arr)

for class_name in os.listdir(original_dir):
    class_dir = os.path.join(original_dir, class_name)
    if not os.path.isdir(class_dir):
        continue
    images = [img for img in os.listdir(class_dir) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)
    split_idx = int(len(images) * (1 - split_ratio))
    train_imgs = images[:split_idx]
    val_imgs = images[split_idx:]

    train_class_dir = os.path.join(split_dir, 'train', class_name)
    val_class_dir = os.path.join(split_dir, 'val', class_name)
    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(val_class_dir, exist_ok=True)

    for img in train_imgs:
        src_img_path = os.path.join(class_dir, img)
        try:
            aug_img = random_orient_augment(src_img_path, IMG_SIZE)
            base, ext = os.path.splitext(img)
            aug_name = f"{base}_aug{ext}"
            save_img(os.path.join(train_class_dir, aug_name), aug_img)
        except Exception as e:
            continue

    for img in val_imgs:
        src_img_path = os.path.join(class_dir, img)
        try:
            aug_img = random_orient_augment(src_img_path, IMG_SIZE)
            base, ext = os.path.splitext(img)
            aug_name = f"{base}_aug{ext}"
            save_img(os.path.join(val_class_dir, aug_name), aug_img)
        except Exception as e:
            continue

print('Train and Test Split is Done!')

Train and Test Split is Done!


In [8]:
train_dir = "/content/plantvillage_split/train"
val_dir = "/content/plantvillage_split/val"
IMG_SIZE = (128, 128)
BATCH_SIZE = 32
INPUT_DIM = 128 * 128 * 3
inputs = tf.keras.Input(shape=(INPUT_DIM,))

def make_conv_autoencoder_dataset(directory, img_size, batch_size):
    ds = tf.keras.utils.image_dataset_from_directory(
        directory,
        labels=None,
        image_size=img_size,
        batch_size=batch_size,
        shuffle=True
    )
    ds = ds.map(lambda x: (x / 255.0, x / 255.0))
    return ds

train_ds = make_conv_autoencoder_dataset(train_dir, IMG_SIZE, BATCH_SIZE)
val_ds = make_conv_autoencoder_dataset(val_dir, IMG_SIZE, BATCH_SIZE)

Found 16504 files.
Found 4134 files.


In [9]:
def build_conv_autoencoder(img_shape):
    inputs = tf.keras.Input(shape=img_shape)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2,2), padding='same')(x)
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2,2), padding='same')(x)
    x = layers.Conv2D(8, (3,3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2,2), padding='same')(x)

    x = layers.Conv2D(8, (3,3), activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2,2))(x)
    outputs = layers.Conv2D(3, (3,3), activation='sigmoid', padding='same')(x)

    autoencoder = tf.keras.Model(inputs, outputs)
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder

autoencoder = build_conv_autoencoder((128, 128, 3))

In [10]:
autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 64, 64, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 32, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 32, 32, 8)      │         2,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 16, 16, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 16, 8)      │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 32, 32, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 32, 32, 32)     │         2,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 128, 128, 3)    │         1,731 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,715 (178.57 KB)

 Trainable params: 45,715 (178.57 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
EPOCHS = 20
history = autoencoder.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ]
)

Epoch 1/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 35s 51ms/step - loss: 0.0179 - val_loss: 0.0092
Epoch 2/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 28s 39ms/step - loss: 0.0090 - val_loss: 0.0084
Epoch 3/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - loss: 0.0084 - val_loss: 0.0081
Epoch 4/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 19s 36ms/step - loss: 0.0081 - val_loss: 0.0078
Epoch 5/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 19s 36ms/step - loss: 0.0079 - val_loss: 0.0077
Epoch 6/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 22s 39ms/step - loss: 0.0077 - val_loss: 0.0076
Epoch 7/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - loss: 0.0076 - val_loss: 0.0074
Epoch 8/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - loss: 0.0075 - val_loss: 0.0074
Epoch 9/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - loss: 0.0074 - val_loss: 0.0074
Epoch 10/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - loss: 0.0074 - val_loss: 0.0072
Epoch 11/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 21s 39ms/step - loss: 0.0073 - val_loss: 0.0072
Epoch 12/20
516/516 ━━━━━━━━━━

In [12]:
def reconstruction_errors(model, X):
    X_pred = model.predict(X, batch_size=32)
    errors = np.mean(np.abs(X - X_pred), axis=1)
    return errors

X_val_tomato_flat = np.concatenate([x for x, _ in val_ds], axis=0)
err_tomato = reconstruction_errors(autoencoder, X_val_tomato_flat)

130/130 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step


In [22]:
print(f"Tomato Error: {err_tomato.mean()}")

Tomato Error: 0.05748232081532478


In [16]:
percentile_85 = np.percentile(err_tomato, 70)
percentile_85

np.float32(0.06864696)

In [13]:
autoencoder.save('/content/autoencoder.h5')
print('Autoencoder saved!')

Autoencoder saved!
